In [1]:
!pip install requests boto3

In [2]:
import requests
import json
import boto3
from datetime import datetime

In [3]:
EIA_API_KEY = "DbCnd9Tz8MSzHzESq4IDN5t6sBnad7KspnBdIjfL"
EIA_API_URL = f"https://api.eia.gov/v2/electricity/rto/daily-region-sub-ba-data/data/?api_key={EIA_API_KEY}&frequency=daily&data[0]=value&start=2020-01-01&end=2022-12-31&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000"

In [4]:
import requests
import time

all_records = []
offset = 0
batch_size = 5000  # Maximum allowed by API

while True:
    paginated_url = f"{EIA_API_URL}&offset={offset}&length={batch_size}"
    print(f"Fetching data from offset: {offset} ...")  # 🟢 Show progress
    
    response = requests.get(paginated_url)
    
    if response.status_code != 200:
        print(f"Error fetching data: {response.status_code}")  # 🛑 If API fails
        break

    batch_data = response.json().get("response", {}).get("data", [])
    
    if not batch_data:
        print("No more data available.")  # 🛑 Stop when there's no data left
        break
    
    all_records.extend(batch_data)
    offset += batch_size  # Move to next batch
    
    print(f"Fetched {len(batch_data)} records, Total: {len(all_records)}")  # 🟢 Track progress
    
    time.sleep(1)  # ⏳ Prevent overloading the API

print(f"Total records fetched: {len(all_records)}")

Fetching data from offset: 0 ...
Fetched 5000 records, Total: 5000
Fetching data from offset: 5000 ...
Fetched 5000 records, Total: 10000
Fetching data from offset: 10000 ...
Fetched 5000 records, Total: 15000
Fetching data from offset: 15000 ...
Fetched 5000 records, Total: 20000
Fetching data from offset: 20000 ...
Fetched 5000 records, Total: 25000
Fetching data from offset: 25000 ...
Fetched 5000 records, Total: 30000
Fetching data from offset: 30000 ...
Fetched 5000 records, Total: 35000
Fetching data from offset: 35000 ...
Fetched 5000 records, Total: 40000
Fetching data from offset: 40000 ...
Fetched 5000 records, Total: 45000
Fetching data from offset: 45000 ...
Fetched 5000 records, Total: 50000
Fetching data from offset: 50000 ...
Fetched 5000 records, Total: 55000
Fetching data from offset: 55000 ...
Fetched 5000 records, Total: 60000
Fetching data from offset: 60000 ...
Fetched 5000 records, Total: 65000
Fetching data from offset: 65000 ...
Fetched 5000 records, Total: 7000

In [13]:
S3_BUCKET_NAME = "ads508-project"
S3_FILE_NAME = f"eia_daily_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

In [14]:
json_data = json.dumps(all_records, indent=2)

In [15]:
s3_client = boto3.client("s3")

s3_client.put_object(
    Bucket=S3_BUCKET_NAME,
    Key=S3_FILE_NAME,
    Body=json_data.encode("utf-8"),  # 🔹 Fix: Encode as bytes
    ContentType="application/json"
)

print(f"✅ Data successfully uploaded to S3: s3://{S3_BUCKET_NAME}/{S3_FILE_NAME}")

✅ Data successfully uploaded to S3: s3://ads508-project/eia_daily_data_20250318_064522.json
